In [1]:
# 반드시 torch/transformers/모듈 import 전에 실행!
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
# 메모리 타이트하면 ↓ 대안
# %env CUBLAS_WORKSPACE_CONFIG=:16:8

!pip install -U deepeval --quiet
# 선택: LlamaIndex/Haystack
# !pip install -U llama-index deepeval-haystack --quiet
!pip install transformers
!pip install -U typing_extensions

env: CUBLAS_WORKSPACE_CONFIG=:4096:8

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from transformers.utils.import_utils import is_accelerate_available


In [3]:
# !pip install jupyterlab jupyter-ai

In [4]:
import sys, os, importlib, inspect

import typing_extensions
print("Executable:", sys.executable)
print("Imported from:", typing_extensions.__file__)
print("Has TypeIs? ->", hasattr(typing_extensions, "TypeIs"))

# Version via package metadata (doesn't rely on __version__ attribute)
try:
    from importlib.metadata import version
    print("Pkg version:", version("typing_extensions"))
except Exception as e:
    print("Pkg version lookup failed:", e)

# Look for local shadowing (files named typing_extensions* in your workspace)
import glob
candidates = glob.glob("./typing_extensions*") + glob.glob("./**/typing_extensions*", recursive=True)
print("Shadowing candidates:", candidates[:20])


Executable: /usr/bin/python
Imported from: /usr/local/lib/python3.11/dist-packages/typing_extensions.py
Has TypeIs? -> True
Pkg version: 4.14.1
Shadowing candidates: []


In [5]:
import sys
!{sys.executable} -m pip install --no-cache-dir -U "typing_extensions==4.12.2" \
    "transformers>=4.41.0" "accelerate>=0.30.0" "bitsandbytes>=0.43.1"


  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.1
    Uninstalling typing_extensions-4.14.1:
      Successfully uninstalled typing_extensions-4.14.1

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
import os, pathlib
os.environ.setdefault("HF_HOME", "/workspace/hf_cache")
os.environ.setdefault("TRANSFORMERS_CACHE", f'{os.environ["HF_HOME"]}/hub')
os.environ.setdefault("HF_DATASETS_CACHE", f'{os.environ["HF_HOME"]}/datasets')
pathlib.Path(os.environ["HF_HOME"]).mkdir(parents=True, exist_ok=True)
pathlib.Path("/workspace/offload").mkdir(parents=True, exist_ok=True)


In [7]:
import os
# 필요 시: 
os.environ["OPENAI_API_KEY"] = 
import gpt_oss_20b_module as gpt  # 앞서 만든 노트북용 모듈 버전 또는 파일
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    AnswerRelevancyMetric,   # 질문 대비 답변 적합성
    FaithfulnessMetric,      # 근거(컨텍스트) 대비 환각 여부
    ToxicityMetric           # 독성/유해성
)


In [8]:
# import sys
# !{sys.executable} -m pip uninstall -y accelerate typing_extensions
# !{sys.executable} -m pip install --no-cache-dir "typing_extensions==4.12.2" "accelerate>=0.30.0"


In [9]:
!pip install -U accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!pip install -U accelerate bitsandbytes


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
import sys
!{sys.executable} -m pip install --no-cache-dir -U "typing_extensions==4.12.2" "accelerate>=0.30.0"
# then restart kernel and verify:
import accelerate, typing_extensions as te
print("accelerate:", accelerate.__version__, "| TypeIs:", hasattr(te, "TypeIs"))



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
accelerate: 1.10.0 | TypeIs: True


In [12]:
import importlib
import gpt_oss_20b_module_patched as gpt

importlib.reload(gpt)
model, tokenizer = gpt.load_model(seed=42)  # accelerate 있으면 device_map="auto", 없으면 단일 디바이스


Loading openai/gpt-oss-20b...
  GPU: NVIDIA RTX 6000 Ada Generation
  Memory: 50.9 GB


MXFP4 quantization requires triton >= 3.4.0 and kernels installed, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model loaded (20.9B params)


In [13]:
import torch, typing_extensions as te, accelerate, bitsandbytes
print("CUDA:", torch.cuda.is_available(), "Device:", torch.cuda.get_device_name(0))
print("TypeIs exists:", hasattr(te, "TypeIs"))


CUDA: True Device: NVIDIA RTX 6000 Ada Generation
TypeIs exists: True


In [14]:
import gpt_oss_20b_module_patched as gpt

# 이미 모델과 토크나이저를 불러왔다면 재로드 불필요
# model, tokenizer = gpt.load_model(seed=42)

prompt = "Explain the theory of relativity in simple terms."

result = gpt.generate(
    prompt=prompt,
    system_message="You are a helpful AI assistant.",
    max_new_tokens=512,
    do_sample=False,   # False → 항상 동일한 답변 (deterministic)
    num_beams=1,
    return_hash=True
)

print("=== Response ===")
print(result["response"])
print("\n=== Metadata ===")
print(f"Input tokens: {result['input_tokens']}")
print(f"Output tokens: {result['output_tokens']}")
print(f"Generation time: {result['generation_time']:.2f}s")
print(f"Tokens/sec: {result['tokens_per_second']:.1f}")


=== Response ===
analysisWe need to explain theory of relativity in simple terms. Likely both special and general relativity. Use analogies, everyday language. Provide key concepts: speed of light constant, time dilation, length contraction, equivalence principle, gravity as curvature of spacetime. Keep simple.assistantfinal**The Theory of Relativity – in plain English**

The theory of relativity is actually two related ideas that Einstein came up with in the early 1900s.  
They change the way we think about space, time, and gravity. Here’s a quick, everyday‑language version of each part.

---

## 1. Special Relativity (1905)

### The two “rules” that changed everything

| Rule | What it means in everyday terms |
|------|---------------------------------|
| **The speed of light is always the same** | No matter how fast you’re moving, if you shine a flashlight, the light will always travel at about 

=== Metadata ===
Input tokens: 91
Output tokens: 200
Generation time: 12.44s
Tokens/sec

In [15]:
prompts = [
    "Explain what PPO is in one sentence.",
    "What is the capital of France?"
]

# 모델 출력 생성
outputs = [gpt.chat(p, system="You are a helpful assistant.", max_new_tokens=72) for p in prompts]

# 기대 출력(optional): 정답 문자열이 있으면 넣어줌(정확 매칭이 아니라 평가 참고값)
expected = [
    "A brief explanation of PPO (Proximal Policy Optimization).",
    "Paris."
]

test_cases = [
    LLMTestCase(
        input=p,
        actual_output=o,
        expected_output=e  # 없으면 None 가능
    )
    for p, o, e in zip(prompts, outputs, expected)
]


In [16]:
rag_question = "Who discovered penicillin and in which year?"
retrieval_context = [
    "Penicillin was discovered in 1928 by Alexander Fleming at St. Mary's Hospital in London."
]
rag_output = gpt.chat(
    f"Context:\n{retrieval_context[0]}\n\nQ: {rag_question}\nA:",
    system="Answer only using the given context.",
    max_new_tokens=64
)

rag_test = LLMTestCase(
    input=rag_question,
    actual_output=rag_output,
    retrieval_context=retrieval_context,
    expected_output="Alexander Fleming discovered penicillin in 1928."
)


In [17]:
# 기본 파라미터로 시작 — 필요 시 임계값/모델 등 조정 가능
metrics = [
    AnswerRelevancyMetric(),  # 문답 적합성
    ToxicityMetric(),         # 독성(참조 불필요, 출력 단독 평가)
]

# 컨텍스트가 있는 케이스에는 Faithfulness도 추가
rag_metrics = metrics + [FaithfulnessMetric()]  # 컨텍스트 기반 환각 검출


## Cell 1 — Environment var

In [ ]:
# Must be set BEFORE importing torch/transformers to satisfy deterministic CuBLAS
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

## Cell 2 — Install

In [ ]:
import sys
!{sys.executable} -m pip install --no-cache-dir -U "typing_extensions==4.12.2" \
    "transformers>=4.41.0" "accelerate>=0.30.0" "bitsandbytes>=0.43.1"

## Cell 3 — Cache/offload dirs

In [ ]:
import os, pathlib
os.environ.setdefault("HF_HOME", "/workspace/hf_cache")
os.environ.setdefault("TRANSFORMERS_CACHE", f'{os.environ["HF_HOME"]}/hub')
os.environ.setdefault("HF_DATASETS_CACHE", f'{os.environ["HF_HOME"]}/datasets')
pathlib.Path(os.environ["HF_HOME"]).mkdir(parents=True, exist_ok=True)
pathlib.Path("/workspace/offload").mkdir(parents=True, exist_ok=True)

## Cell 4 — Load module and model

In [ ]:
import importlib
import gpt_oss_20b_module_patched as gpt

importlib.reload(gpt)  # optional if you’re editing the file in-place
model, tokenizer = gpt.load_model(seed=42)


## Cell 5 — Inference

In [ ]:
prompt = "Explain the theory of relativity in simple terms."

result = gpt.generate(
    prompt=prompt,
    system_message="You are a helpful AI assistant.",
    max_new_tokens=200,
    do_sample=False,   # deterministic
    num_beams=1,
    return_hash=True
)

print("=== Response ===")
print(result["response"])
print("\n=== Metadata ===")
print(f"Input tokens: {result['input_tokens']}")
print(f"Output tokens: {result['output_tokens']}")
print(f"Generation time: {result['generation_time']:.2f}s")
print(f"Tokens/sec: {result['tokens_per_second']:.1f}")
